In [1]:
import numpy as np
import pandas as pd
import itertools
from __future__ import division
import geoplotlib as glp
from geoplotlib.utils import BoundingBox, DataAccessObject
from sklearn import cluster

%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
filePath = 'datasets/NYPD_Motor_Vehicle_Collisions.csv'

collisions = pd.read_csv(filePath)

In [3]:
mask = ((collisions['VEHICLE TYPE CODE 1'] == 'PASSENGER VEHICLE') & (pd.notnull(collisions.LOCATION)))
#mask = (pd.notnull(collisions.LOCATION))

In [4]:
cols_pass_veh = collisions.loc[mask]
print "Total %d, passenger vehicles: %d" % (len(collisions), len(cols_pass_veh))

Total 769054, passenger vehicles: 347331


In [5]:
def knn(k, dataset, colums):
    md = cluster.KMeans(n_clusters=k).fit(dataset[colums])
    return md.predict(dataset[colums]),md.cluster_centers_

In [6]:
k = 10
#md = KMeans(n_clusters=k).fit(cols_pass_veh[['LONGITUDE', 'LATITUDE']])

knns, centoid = knn(k, collisions.loc[mask], ['LONGITUDE', 'LATITUDE'])

result = pd.DataFrame(cols_pass_veh)# = result
result['K-mean'] = knns


print len(cols_pass_veh) 

result.to_csv('datasets/clusters/all.csv', sep=',')

347331


In [7]:
import geoplotlib as gpl

def get_spaced_colors(n):
    max_value = 16581375 #255**3
    interval = int(max_value / n)
    colors = [hex(I)[2:].zfill(6) for I in range(0, max_value, interval)]
    
    return [[int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16), 255] for i in colors]

colormap = get_spaced_colors(k)

def coords(k):
    lat = result[result['K-mean'] == k].LATITUDE.values
    lon = result[result['K-mean'] == k].LONGITUDE.values
    
    return lat,lon
    
for i in range(0,k):
    lat, lon = coords(i)

    data = {'lon': lon, 'lat': lat}
    gpl.dot(data, color=colormap[i])

gpl.inline()